In [1]:
import pandas as pd
import dbconfig as dbConf

In [2]:
def GetPatten():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    #print("접속완료")

    sql = ("""
           select stan
                , 3cnt
                , 5cnt
                , 10cnt
                , 30cnt
             from in_list_2
            order by 1 desc
            limit 15
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [3]:
def Update_in_list(max_val):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    
    update_in_list = (
                    """
                    replace into in_list
                    select {0} as seq
                         , 3      as val
                         , num    as num
                         , cnt_30 as cnt
                      from (
                            select a.num
                                 , ifnull(a.cnt_30,0) as cnt_30
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-9  and {0}) b where a.num = b.num),'') as '10r'
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-4  and {0}) b where a.num = b.num),'') as '5r'
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-2  and {0}) b where a.num = b.num),'') as '3r'
                                 , ifnull((select 'O' from temp2 where seq = {0} and a.num = num),'')                                                       as 'collect'
                              from (
                                    select num 
                                         , sum(case when seq between {0}-29 and {0} then 1 end) cnt_30
                                      from temp2
                                     group by num 
                                   ) a 
                            ) b
                     where 1=1 
                      and 3r = 'O'
                    union all
                    select {0} as seq
                         , 5      as val
                         , num    as num
                         , cnt_30 as cnt
                      from (
                            select a.num
                                 , ifnull(a.cnt_30,0) as cnt_30
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-9  and {0}) b where a.num = b.num),'') as '10r'
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-4  and {0}) b where a.num = b.num),'') as '5r'
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-2  and {0}) b where a.num = b.num),'') as '3r'
                                 , ifnull((select 'O' from temp2 where seq = {0} and a.num = num),'')                                                       as 'collect'
                              from (
                                    select num 
                                         , sum(case when seq between {0}-29 and {0} then 1 end) cnt_30
                                      from temp2
                                     group by num 
                                   ) a 
                            ) b
                     where 1=1 
                      and 3r <> 'O' and 5r = 'O'
                    union all
                    select {0} as seq
                         , 10     as val
                         , num    as num
                         , cnt_30 as cnt
                      from (
                            select a.num
                                 , ifnull(a.cnt_30,0) as cnt_30
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-9  and {0}) b where a.num = b.num),'') as '10r'
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-4  and {0}) b where a.num = b.num),'') as '5r'
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-2  and {0}) b where a.num = b.num),'') as '3r'
                                 , ifnull((select 'O' from temp2 where seq = {0} and a.num = num),'')                                                       as 'collect'
                              from (
                                    select num 
                                         , sum(case when seq between {0}-29 and {0} then 1 end) cnt_30
                                      from temp2
                                     group by num 
                                   ) a 
                            ) b
                     where 1=1 
                      and 3r <> 'O' and 5r <> 'O' and 10r = 'O'
                    union all
                    select {0} as seq
                         , 30     as val
                         , num    as num
                         , cnt_30 as cnt
                      from (
                            select a.num
                                 , ifnull(a.cnt_30,0) as cnt_30
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-9  and {0}) b where a.num = b.num),'') as '10r'
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-4  and {0}) b where a.num = b.num),'') as '5r'
                                 , ifnull((select 'O' from (select distinct num from temp2 where seq between {0}-2  and {0}) b where a.num = b.num),'') as '3r'
                                 , ifnull((select 'O' from temp2 where seq = {0} and a.num = num),'')                                                       as 'collect'
                              from (
                                    select num 
                                         , sum(case when seq between {0}-29 and {0} then 1 end) cnt_30
                                      from temp2
                                     group by num 
                                   ) a 
                            ) b
                     where 1=1 
                      and 3r <> 'O' and 5r <> 'O' and 10r <> 'O'
                    order by 1,2,4 desc
                    """.format(max_val)
                  )       
    zeroDb.update(update_in_list)
    #print(update_in_list)  
    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    zeroDb.commit()
    zeroDb.closedb()

In [4]:
# Update_in_list_2
def Update_in_list_2():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    
    update_in_list_2 = (
                    """
                    replace into in_list_2
                    select seq as stan
                         , sum(case when val = 3  then 1 else 0 end) as 3cnt
                         , sum(case when val = 5  then 1 else 0 end) as 5cnt
                         , sum(case when val = 10 then 1 else 0 end) as 10cnt
                         , sum(case when val = 30 then 1 else 0 end) as 30cnt
                         , '' as bin
                         , (select seq from temp where seq = a.seq + 1) as seq
                         , (select n1  from temp where seq = a.seq + 1) as seq
                         , (select n2  from temp where seq = a.seq + 1) as seq
                         , (select n3  from temp where seq = a.seq + 1) as seq
                         , (select n4  from temp where seq = a.seq + 1) as seq
                         , (select n5  from temp where seq = a.seq + 1) as seq
                         , (select n6  from temp where seq = a.seq + 1) as seq
                      from in_list a
                     where 1=1 
                       and a.num in (
                                     select b.num
                                       from temp2 b 
                                      where b.seq = a.seq + 1
                                        and b.num = a.num
                                    ) 
                     group by seq
                     order by 1 desc
                    ;
                    """
                  )       
    zeroDb.update(update_in_list_2)
    #print(update_in_list_2)  
    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    zeroDb.commit()
    zeroDb.closedb()

In [5]:
def GetNumber(max_val, f1, f2, f3, f4, val):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = ("""
             select sum(case when row_num = 1 then num end) as n1
                  , sum(case when row_num = 2 then num end) as n2
                  , sum(case when row_num = 3 then num end) as n3
                  , sum(case when row_num = 4 then num end) as n4
                  , sum(case when row_num = 5 then num end) as n5
                  , sum(case when row_num = 6 then num end) as n6
                  , '' as bin
                  , (select stan  from in_list_2 where stan = {5}) as stan
                  , (select 3cnt  from in_list_2 where stan = {5}) as cnt3
                  , (select 5cnt  from in_list_2 where stan = {5}) as cnt5
                  , (select 10cnt from in_list_2 where stan = {5}) as cnt10
                  , (select 30cnt from in_list_2 where stan = {5}) as cnt30
               from (
             select num
                  , @rownum:=@rownum+1 as row_num
               from (
                    select seq, num   
                      from (select 3 as seq, num
                              from in_list 
                             where seq = {0} 
                               and val = 3 
                             order by rand() 
                             limit {1}
                           ) as t3
                    union all
                    select seq, num   
                      from (select 5 as seq, num
                              from in_list 
                             where seq = {0}
                               and val = 5 
                             order by rand() 
                             limit {2}
                           ) as t5 
                    union all
                    select seq, num   
                      from (select 10 as seq, num
                              from in_list 
                             where seq = {0}
                               and val = 10
                             order by rand() 
                             limit {3}
                           ) as t10
                    union all
                    select seq, num   
                    from (select 30 as seq, num
                            from in_list 
                           where seq = {0}
                             and val = 30 
                           order by rand() 
                           limit {4}
                         ) as t30
                     order by num 
                 ) as a
                 , (SELECT @rownum:=0) TMP
                 ) as b 
             ;
           """.format(max_val, f1, f2, f3, f4, val)
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [6]:
def GetNumber_one(max_val, f1, f2, f3, f4):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = ("""
             select sum(case when row_num = 1 then num end) as n1
                  , sum(case when row_num = 2 then num end) as n2
                  , sum(case when row_num = 3 then num end) as n3
                  , sum(case when row_num = 4 then num end) as n4
                  , sum(case when row_num = 5 then num end) as n5
                  , sum(case when row_num = 6 then num end) as n6
                  , '' as bin
                  , sum(case when row_num = 1 then seq end) as n1_seq
                  , sum(case when row_num = 2 then seq end) as n2_seq
                  , sum(case when row_num = 3 then seq end) as n3_seq
                  , sum(case when row_num = 4 then seq end) as n4_seq
                  , sum(case when row_num = 5 then seq end) as n5_seq
                  , sum(case when row_num = 6 then seq end) as n6_seq
               from (
             select num
                  , @rownum:=@rownum+1 as row_num
                  , seq
               from (
                    select seq, num   
                      from (select 3 as seq, num
                              from in_list 
                             where seq = {0} 
                               and val = 3 
                             order by rand() 
                             limit {1}
                           ) as t3
                    union all
                    select seq, num   
                      from (select 5 as seq, num
                              from in_list 
                             where seq = {0}
                               and val = 5 
                             order by rand() 
                             limit {2}
                           ) as t5 
                    union all
                    select seq, num   
                      from (select 10 as seq, num
                              from in_list 
                             where seq = {0}
                               and val = 10
                             order by rand() 
                             limit {3}
                           ) as t10
                    union all
                    select seq, num   
                    from (select 30 as seq, num
                            from in_list 
                           where seq = {0}
                             and val = 30 
                           order by rand() 
                           limit {4}
                         ) as t30
                     order by num 
                 ) as a
                 , (SELECT @rownum:=0) TMP
                 ) as b 
             ;
           """.format(max_val, f1, f2, f3, f4)
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

### 첫번째 부터 ~ 마지막까지 건수

In [7]:
# 첫번째 부터 ~ 마지막까지 당첨 건수 넣기
for i in range(998, 1000):
    Update_in_list(i)

### in_list 업데이트

In [15]:
# in_list 업데이트 
Update_in_list(stan[1]+1)

In [16]:
# in_list_2 업데이트 
Update_in_list_2()

### 최신회차 리스트

In [17]:
# 최신회차 뽑기
df = GetPatten()

stan  = list(range(0,1))
cnt3  = list(range(0,1))
cnt5  = list(range(0,1))
cnt10 = list(range(0,1))
cnt30 = list(range(0,1))

for df_stan, df_cnt3, df_cnt5, df_cnt10, df_cnt30 in zip(df['stan'], df['3cnt'], df['5cnt'], df['10cnt'], df['30cnt']):
    stan.append(int(df_stan))
    cnt3.append(int(df_cnt3))
    cnt5.append(int(df_cnt5))
    cnt10.append(int(df_cnt10))
    cnt30.append(int(df_cnt30))

### 최종 결과

In [18]:
# 최신 패턴으로 결과 뽑기
data = GetNumber(stan[1]+1, cnt3[1], cnt5[1], cnt10[1], cnt30[1], stan[1])

for i in range(2, 16):
    data = data.append(GetNumber(stan[1]+1, cnt3[i], cnt5[i], cnt10[i], cnt30[i], stan[i]), ignore_index = True)
    
data

,n1,n2,n3,n4,n5,n6,bin,stan,cnt3,cnt5,cnt10,cnt30
0,1,5,7,15,18,39,,1001,0,2,3,1
1,9,11,17,20,35,37,,1000,3,1,1,1
2,1,5,16,17,23,29,,999,1,1,2,2
3,13,23,26,28,33,35,,998,2,2,0,2
4,3,4,12,18,24,44,,997,1,2,3,0
5,2,3,18,28,32,41,,996,2,3,0,1
6,5,11,19,31,32,33,,995,3,0,1,2
7,13,15,16,26,28,45,,994,1,2,2,1
8,5,7,13,29,33,34,,993,1,1,2,2
9,5,16,27,31,40,42,,992,1,0,2,3


In [19]:
# 최종 결과 한개씩 뽑기
GetNumber_one(stan[1]+1, 3, 2, 1, 0)

,n1,n2,n3,n4,n5,n6,bin,n1_seq,n2_seq,n3_seq,n4_seq,n5_seq,n6_seq
0,1,10,13,17,29,32,,5,3,5,3,10,3


### 한 패턴으로

In [20]:
# 필요 갯수 및 패턴 수 
# 합 = 6
round_cnt = 5 

cnt3_val  = 3
cnt5_val  = 2
cnt10_val = 1
cnt30_val = 0

same_data = GetNumber_one(stan[1]+1, 0, 0, 0, 0)

for i in range(0, round_cnt):
    same_data = same_data.append(GetNumber_one(stan[1], cnt3_val, cnt5_val, cnt10_val, cnt30_val), ignore_index = True)

same_data

,n1,n2,n3,n4,n5,n6,bin,n1_seq,n2_seq,n3_seq,n4_seq,n5_seq,n6_seq
0,None,None,None,None,None,None,,None,None,None,None,None,None
1,1,2,14,24,33,44,,3,3,3,5,10,5
2,1,3,13,24,26,32,,3,3,5,5,10,3
3,2,3,4,7,8,26,,3,3,5,5,3,10
4,3,17,18,22,27,45,,3,5,3,3,10,5
5,8,13,17,19,26,32,,3,5,5,3,10,3


In [21]:
stan[1]

1001